## 네이버뉴스 언론사/일자별 동적 웹크롤링

- 실행 전 라이브러리 설치
- pip install webdriver-manager

- 코드 상단 📌 타깃 URL의 각 요소 수정해서 그대로 실행
- 예시 URL : https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=009&date=20251218
- 예시 URL에서 OID: 009, DATE: 20251218
- 👉 "oid=009&date=20251218.csv"로 본 파일과 같은 위치에 저장됨. (oid가 언론사 url 코드)

In [ ]:
import time
import pandas as pd
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import os

# ====================================================
# 📌 [설정] 여기만 수정하세요
# ====================================================
TARGET_OID = "018"        # 언론사 코드
START_DATE = "20251212"   # 시작 날짜 (YYYYMMDD)
END_DATE   = "20251220"   # 종료 날짜 (YYYYMMDD)
# 언론사 코드: 
# 008: 머니투데이, 009: 매일경제, 011: 서울경제, 014: 파이낸셜뉴스, 015: 한국경제, 
# 016: 헤럴드경제, 018: 이데일리, 123: 조세일보, 277: 아시아경제, 366: 조선비즈, 648 비즈워치
# ====================================================


# [보조 함수] 날짜 텍스트 표준화
def clean_date(date_str):
    try:
        date_str = str(date_str).replace("기사입력", "").replace("입력", "").strip()
        is_pm = "오후" in date_str
        date_str = date_str.replace("오전", "").replace("오후", "").strip()
        dt = datetime.strptime(date_str, "%Y.%m.%d. %H:%M")
        if is_pm and dt.hour != 12: dt = dt.replace(hour=dt.hour + 12)
        elif not is_pm and dt.hour == 12: dt = dt.replace(hour=0)
        return dt.strftime("%Y-%m-%d %H:%M:%S")
    except:
        return date_str

# 1. 드라이버 설정
def set_chrome_driver():
    chrome_options = Options()
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

# 2. 기사 상세 내용 추출
def extract_article_info(driver, url):
    try:
        driver.get(url)
        time.sleep(0.3)

        if "entertain.naver.com" in driver.current_url: return None
        if "sports.naver.com" in url: continue

        try:
            WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#title_area > span")))
        except:
            return None

        try: title = driver.find_element(By.CSS_SELECTOR, "#title_area > span").text
        except: title = "제목 없음"

        try:
            dic_area = driver.find_element(By.CSS_SELECTOR, "#dic_area")
            driver.execute_script("""
                var element = arguments[0];
                var dirts = element.querySelectorAll(".img_desc, .media_end_summary"); 
                for (var i = 0; i < dirts.length; i++) { dirts[i].remove(); }
            """, dic_area)
            content = dic_area.text.replace("\n", " ").strip()
        except: content = "본문 없음"

        try:
            date_element = driver.find_element(By.CSS_SELECTOR, "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
            raw_date = date_element.text
        except:
            try: raw_date = driver.find_element(By.CSS_SELECTOR, ".media_end_head_info_datestamp span").text
            except: raw_date = "날짜 없음"
        
        clean_date_str = clean_date(raw_date)
        print(f"   [수집] {title} ({clean_date_str})")
        
        return {"날짜": clean_date_str, "제목": title, "본문": content, "링크": url}

    except: return None

# 3. 특정 날짜 하루치 크롤링 함수
def crawl_one_day(driver, oid, date_str):
    target_url = f"https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid={oid}&date={date_str}"
    print(f"\n📅 [날짜 시작] {date_str} 수집 시작 -> {target_url}")
    
    driver.get(target_url)
    time.sleep(1)
    
    daily_data = []
    page_count = 1
    
    while True:
        print(f"   페이지 탐색 중: {page_count}...")
        
        # URL 수집
        article_urls = []
        selectors = [
            "#main_content > div.list_body.newsflash_body > ul.type06_headline > li dl > dt:not(.photo) > a",
            "#main_content > div.list_body.newsflash_body > ul.type06 > li dl > dt:not(.photo) > a"
        ]
        for sel in selectors:
            links = driver.find_elements(By.CSS_SELECTOR, sel)
            for link in links:
                url = link.get_attribute("href")
                if url: article_urls.append(url)
        
        # 상세 순회
        list_window = driver.current_window_handle
        for url in article_urls:
            if "entertain.naver.com" in url: continue
            if "sports.naver.com" in url: continue
            
            driver.execute_script("window.open('');")
            driver.switch_to.window(driver.window_handles[-1])
            
            data = extract_article_info(driver, url)
            if data: daily_data.append(data)
            
            driver.close()
            driver.switch_to.window(list_window)
            
        # 페이징
        try:
            paging_area = driver.find_element(By.CSS_SELECTOR, "#main_content > div.paging")
            current_num = int(paging_area.find_element(By.TAG_NAME, "strong").text)
            next_num = current_num + 1
            
            # 숫자 버튼
            try:
                next_btn = paging_area.find_element(By.XPATH, f".//a[text()='{next_num}']")
                next_btn.click()
                page_count += 1
                time.sleep(1.5)
                continue
            except: pass
            
            # 다음 그룹 버튼
            try:
                next_group = paging_area.find_element(By.CSS_SELECTOR, "a.next")
                next_group.click()
                page_count += 1
                time.sleep(1.5)
                continue
            except:
                print(f"   ✅ {date_str} 수집 종료 (총 {len(daily_data)}건)")
                break
        except:
            break

    return daily_data

# 4. 메인 실행 함수
def main():
    driver = set_chrome_driver()
    
    # 날짜 범위 계산
    start_dt = datetime.strptime(START_DATE, "%Y%m%d")
    end_dt = datetime.strptime(END_DATE, "%Y%m%d")
    delta = (end_dt - start_dt).days + 1
    
    try:
        # 날짜 루프
        for i in range(delta):
            current_date = start_dt + timedelta(days=i)
            date_str = current_date.strftime("%Y%m%d")
            
            # 1) 하루치 데이터 수집
            one_day_data = crawl_one_day(driver, TARGET_OID, date_str)
            
            # 2) [수정됨] 하루치 수집 즉시 개별 파일 저장
            if one_day_data:
                df = pd.DataFrame(one_day_data)
                df = df[['날짜', '제목', '본문', '링크']]
                
                # 파일명: oid=009&date=20251215.csv
                filename = f"oid={TARGET_OID}&date={date_str}.csv"
                
                # 현재 폴더에 저장
                df.to_csv(filename, index=False, encoding='utf-8-sig')
                print(f"   💾 저장 완료: {filename} ({len(df)}개 기사)\n")
            else:
                print(f"   ⚠️ {date_str} 데이터가 없어 저장하지 않습니다.\n")
            
    except KeyboardInterrupt:
        print("\n[중단] 작업을 멈춥니다.")
    finally:
        driver.quit()
        print("🚀 모든 작업이 종료되었습니다.")

if __name__ == "__main__":
    main()


📅 [날짜 시작] 20251201 수집 시작 -> https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=018&date=20251201
   페이지 탐색 중: 1...
   [수집] "그 사람 車 아닌데"...1억 BMW '활활', 알고보니 (2025-12-01 23:03:00)
   [수집] ‘청산 위기’ 홈플러스에…노조, 물·소금 끊고 아사 단식 돌입 (2025-12-01 22:58:00)
   [수집] 폐지 줍는 할머니에 ‘무제한 카드’ 준 유튜버…"85년 살며 처음" (2025-12-01 22:48:00)
   [수집] 또 자택서 쓰러진 ‘쿠팡 새벽배송’ 기사…노조 측, 진상규명 촉구 (2025-12-01 22:44:00)
   [수집] 조건만남 응한 남성에게 강도질 지시…10대 구속 (2025-12-01 22:31:00)
   [수집] `민중기 특검`의 강압 수사, 故 양평군 공무원 유서에 써 있었다(종합) (2025-12-01 22:28:00)
   [수집] '음주운전 3회 시 가중처벌' 위헌 논란…헌재 "만장일치 합헌" (2025-12-01 22:15:00)
   [수집] 기술혁신 3대 학회, 바이오·AI 대응방안 논의 (2025-12-01 21:57:00)
   [수집] 홍대입구역 승강장서 화재…역무원 자체 진화로 인명피해 없어 (2025-12-01 21:45:00)
   [수집] "한국 여행 즐거웠습니다"…모금함에 110만원 남긴 日여행객 (2025-12-01 21:41:00)
   [수집] '로봇위성 전문' 워커린스페이스, 90억원 규모 프리A 투자 유치 (2025-12-01 21:40:00)
   [수집] ‘눈물’의 엔비디아 주식 매각…손정의 “한주도 팔고 싶지 않았다” (2025-12-01 21:27:00)
   [수집] 강유정 “李대통령, 장난꾸러기에 막내 기질…혼난 적 없어” (2025-12-01 21:25:00)
   [수집] ‘히틀러’ 연상시킨 독일 극우당 청년 정치인 연설 